# Projeto 3
### por: Gabriel Pizzighini, Pedro Azambuja, Luigi Corradi


#### importando blibiotecas 

In [1]:
import plotly as py
from plotly.graph_objs import*
import pandas as pd
py.offline.init_notebook_mode(connected=True)

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as sp
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.datasets.species_distributions import construct_grids
from sklearn.datasets import fetch_species_distributions
from sklearn.cluster import KMeans
from matplotlib import style

#### Importando o Database

In [3]:
dt = pd.read_csv("globalterrorismd.csv", encoding ='latin1')
dt.head()

C:\Users\Faculdade\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning:

Columns (4,6,31,33,53,61,62,63,76,79,90,92,94,96,114,115,121) have mixed types. Specify dtype option on import or set low_memory=False.



,eventid,iyear,imonth,iday,approxdate,extended,resolution,country,country_txt,region,...,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
0,197000000001,1970,7,2,NaN,0,NaN,58,Dominican Republic,2,...,NaN,NaN,NaN,NaN,PGIS,0,0,0,0,NaN
1,197000000002,1970,0,0,NaN,0,NaN,130,Mexico,1,...,NaN,NaN,NaN,NaN,PGIS,0,1,1,1,NaN
2,197001000001,1970,1,0,NaN,0,NaN,160,Philippines,5,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
3,197001000002,1970,1,0,NaN,0,NaN,78,Greece,8,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
4,197001000003,1970,1,0,NaN,0,NaN,101,Japan,4,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN


In [4]:
# lipando o Database
dt = dt.loc[:, ['iyear','imonth','iday','country','gname',
'country_txt',
'region_txt',
'region',
'city',
'latitude',
'longitude',
'success',
'attacktype1',
'attacktype1_txt',
'targtype1',
'targtype1_txt',
'targsubtype1',
'targsubtype1_txt',
 'weaptype1',
 'weaptype1_txt',
 'weapsubtype1',
 'weapsubtype1_txt',
 'weaptype2',
 'weaptype2_txt',
 'weapsubtype2',
 'weapsubtype2_txt',
 'nkill',
 'nwound',]]
dt= dt[(dt.region_txt=="Eastern Europe")|(dt.region_txt=="Western Europe")]
dt = dt[dt.iyear >= 2000]
#dt = dt[dt.country_txt != 'Russia']
dt.nkill = dt.nkill.fillna(0)
dt.nwound = dt.nwound.fillna(0)

#### Criando o calculador de probabilidade 

In [5]:
ataques = []
for i in dt.attacktype1_txt:
    if i not in ataques:
        ataques.append(i)

paises = []
for i in dt.country_txt:
    if i not in paises:
        paises.append(i)

In [6]:
def bayes_attack(x,y):
        dt_novo = dt[dt.attacktype1_txt == y]
        total = len(dt_novo)
        P = len(dt_novo[dt_novo.country_txt == x])/total
        return P

In [7]:
dict_p ={}
for i in ataques:
    dict_p[i] = {}
    for k in paises:
        ind = 'P({0}|{1})'.format(k,i)
        P = bayes_attack(k,i)
        #print('P({0}|{1}) = {2}'.format(k,i,P)
        dict_p[i][k] = bayes_attack(k,i)

In [8]:
df = pd.read_csv('all.csv')
df.head()

,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,region-code,sub-region-code
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,142.0,34.0
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,150.0,154.0
2,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,150.0,39.0
3,Algeria,DZ,DZA,12,ISO 3166-2:DZ,Africa,Northern Africa,2.0,15.0
4,American Samoa,AS,ASM,16,ISO 3166-2:AS,Oceania,Polynesia,9.0,61.0


In [9]:
colo_colum = pd.DataFrame(dict_p)

#Encontrando valores dos códigos
paises = colo_colum.index
p_codes = []
for i in range(len(df)):
    for e in paises:
        if e == df.loc[i][0]:
            p_codes.append(df.loc[i][1])
p_codes = ['ALB', 'AUT', 'BLR', 'BEL','BIH', 'BGR', 'HRV',
           'CYP', 'CZE', 'DNK', 'EST', 'FIN', 'FRA', 'DEU',
           'GRC', 'HUN', 'ISL', 'IRL', 'ITA','RKS', 'LVA',
           'MKD', 'MLT','MDA', 'MNE', 'NLD', 'NOR', 'POL',
           'PRT', 'ROU','RUS', 'SRB','SCG','SVK', 'SVN',
           'ESP', 'SWE', 'CHE', 'UKR','GBR']

#Retirando a yugoslavia
colo_colum = colo_colum.iloc[:-1]

#Criando coluna : "CODE"
colo_colum["CODE"] = p_codes
colo_colum

,Armed Assault,Assassination,Bombing/Explosion,Facility/Infrastructure Attack,Hijacking,Hostage Taking (Barricade Incident),Hostage Taking (Kidnapping),Unarmed Assault,Unknown,CODE
Albania,0.000000,0.003546,0.002932,0.000000,0.000000,0.000000,0.005882,0.009615,0.000000,ALB
Austria,0.001467,0.000000,0.000000,0.014300,0.000000,0.000000,0.000000,0.009615,0.000000,AUT
Belarus,0.000000,0.000000,0.001599,0.002043,0.000000,0.000000,0.000000,0.009615,0.000000,BLR
Belgium,0.002201,0.000000,0.001333,0.006129,0.000000,0.000000,0.000000,0.076923,0.000000,BEL
Bosnia-Herzegovina,0.003668,0.010638,0.005330,0.003064,0.000000,0.000000,0.000000,0.009615,0.010526,BIH
Bulgaria,0.000000,0.007092,0.002932,0.002043,0.000000,0.000000,0.000000,0.019231,0.000000,BGR
Croatia,0.000000,0.003546,0.002665,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,HRV
Cyprus,0.000000,0.003546,0.006130,0.001021,0.000000,0.000000,0.000000,0.000000,0.000000,CYP
Czech Republic,0.000000,0.003546,0.001866,0.006129,0.000000,0.000000,0.000000,0.038462,0.000000,CZE
Denmark,0.002935,0.003546,0.000267,0.002043,0.000000,0.000000,0.000000,0.000000,0.000000,DNK


## Graficos para analise:

Escolha um dos seguintes ataques:
* Bombing/Explosion
* Armed Assault
* Facility/Infrastructure Attack
* Assassination
* Unarmed Assault
* Unknown
* Hostage Taking (Kidnapping)
* Hijacking
* Hostage Taking (Barricade Incident)

**Atenção!** porfavor copiar **exatamente** o texto.

In [ ]:
coluna_escolhida = input("Qual o tipo de ataque: ")
if coluna_escolhida in ataques:
    data = [ dict(
        type = 'choropleth',
        locations = colo_colum['CODE'],
        z = colo_colum[coluna_escolhida],
        text = colo_colum.index,
        colorscale = [[0,"rgb(5, 10, 172)"],[0.35,"rgb(40, 60, 190)"],[0.5,"rgb(70, 100, 245)"],\
            [0.6,"rgb(90, 120, 245)"],[0.7,"rgb(106, 137, 247)"],[1,"rgb(220, 220, 220)"]],
        autocolorscale = False,
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            autotick = False,
            tickprefix = '- ',
            title = 'Probability'),
      ) ]

    layout = dict(
        title = 'Terrorism for {0}(2000-2015)'.format(coluna_escolhida),
        geo = dict(
            showframe = False,
            showcoastlines = False,
            projection = dict(
                type = 'Mercator'
            )
        )
    )

    fig = dict( data=data, layout=layout )
    py.offline.iplot( fig, validate=False, filename='d3-world-map' )
else :
    print("Desculpe nao conseguimos encontrar seu tipo de ataque, poderia tentar de novo?")
    return